
# Classifications of in-vivo Gastral Images - Training

In [17]:
import os
import random
from shutil import copyfile

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

## Constants
We need to specify a set of "constants" (Python does not have constants, but it really *should* be constants ;-)) that depend on the data location of the machine that runs the code.
Note that you can get the data from https://rdm.inesctec.pt/dataset/nis-2018-003.

In [3]:
label_path = '/Users/timotheuskampik/Downloads/labels.txt' # Path to labels file
data_path = '/Users/timotheuskampik/Downloads/dataset/Set_1/A' # Path to data folder
working_path = '/Users/timotheuskampik/Downloads/training' # Path for (to-be-prepared) training data

## Data Preparation
First, we need to prepare our data. We split data and labels into train and validation sets (randomly assigned) and, for each set, separate *bleeding* and *non-bleeding* images.

In [19]:
training_path = f'{working_path}/training'
validation_path = f'{working_path}/validation'
validation_split = 0.1

# read in labels
labels_file = open(label_path, 'r')
labels = labels_file.read().splitlines()

# get data set length
_, _, files = next(os.walk(data_path))
data_length = len(files)

print(f'Data length: {data_length}; label length: {len(labels)}')

# create directory structure
if not os.path.exists(training_path):
    os.makedirs(training_path)
if not os.path.exists(validation_path):
    os.makedirs(validation_path)

if not os.path.exists(f'{training_path}/bleeding'):
    os.makedirs(f'{training_path}/bleeding')
if not os.path.exists(f'{training_path}/nonbleeding'):
    os.makedirs(f'{training_path}/nonbleeding')
if not os.path.exists(f'{validation_path}/bleeding'):
    os.makedirs(f'{validation_path}/bleeding')
if not os.path.exists(f'{validation_path}/nonbleeding'):
    os.makedirs(f'{validation_path}/nonbleeding')

# copy files into directory structure
index = 0
for _, _, files in os.walk(data_path):
    for file in files:
        base_path = training_path
        if random.uniform(0, 1) < validation_split: # assign pseudo-randomly to training or validation set
            base_path = validation_path
        if labels[index] == '0':
            copyfile(os.path.join(data_path, file), f'{base_path}/nonbleeding/{file}') 
        else:
            copyfile(os.path.join(data_path, file), f'{base_path}/bleeding/{file}')
        index = index + 1

_, _, files = next(os.walk(data_path))
training_data_length = len(files)

_, _, files = next(os.walk(data_path))
validation_data_length = len(files)

Data length: 3295; label length: 3295


## Training
Now, we save the model and save it to disk. For this, we use this Keras code as the blue print: https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d.

In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

nb_train_samples = training_data_length
nb_validation_samples = validation_data_length
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    training_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Found 2959 images belonging to 2 classes.
Found 336 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
  6/205 [..............................] - ETA: 1:50 - loss: 0.6960 - acc: 0.5729